In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sb

# **Try to overfit CV pls !**    Upvote if you fork/liked it, Thanks !

Credit : Tokenizer training + TFIDF + RIDGE [LB 0.860] by Adriano Passos
https://www.kaggle.com/coldfir3/tokenizer-training-tfidf-ridge-lb-0-860

In [ ]:
import optuna

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from tqdm.auto import tqdm

TRAIN_DATA_PATH = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
VALID_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv"
TEST_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"

df_train2 = pd.read_csv(TRAIN_DATA_PATH)
df_valid2 = pd.read_csv(VALID_DATA_PATH)
df_test2 = pd.read_csv(TEST_DATA_PATH)


#optuna Version
from optuna.integration import LightGBMPruningCallback
import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
def objective(trial):
    # Parameters Network
    cat_mtpl = {

        "obscene": trial.suggest_float("obscene", 0.01, 2),
        "toxic": trial.suggest_float("toxic", 0.01, 2),
        "threat": trial.suggest_float("threat", 0.01, 2),
        "insult": trial.suggest_float("insult", 0.01, 2),
        "severe_toxic": trial.suggest_float("severe_toxic", 0.01, 2),
        'identity_hate': trial.suggest_float("identity_hate", 0.01, 2),
    }
    
    for category in cat_mtpl:
        df_train2[category] = df_train2[category] * cat_mtpl[category]

    df_train2['score'] = df_train2.loc[:, 'toxic':'identity_hate'].mean(axis=1)

    df_train2['y'] = df_train2['score']

    min_len = (df_train2['y'] > 0).sum()  # len of toxic comments
    df_y0_undersample = df_train2[df_train2['y'] == 0].sample(n=min_len, random_state=41)  # take non toxic comments
    df_train_new = pd.concat([df_train2[df_train2['y'] > 0], df_y0_undersample])  # make new df
    from tokenizers import (
        decoders,
        models,
        normalizers,
        pre_tokenizers,
        processors,
        trainers,
        Tokenizer,
    )
    raw_tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
    raw_tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)
    raw_tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
    special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
    trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
    from datasets import Dataset

    dataset = Dataset.from_pandas(df_train_new[['comment_text']])

    def get_training_corpus():
        for i in range(0, len(dataset), 1000):
            yield dataset[i : i + 1000]["comment_text"]

    raw_tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

    from transformers import PreTrainedTokenizerFast

    tokenizer = PreTrainedTokenizerFast(
        tokenizer_object=raw_tokenizer,
        unk_token="[UNK]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        sep_token="[SEP]",
        mask_token="[MASK]",
    )
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.linear_model import Ridge

    def dummy_fun(doc):
        return doc

    labels = df_train_new['y']
    comments = df_train_new['comment_text']
    tokenized_comments = tokenizer(comments.to_list())['input_ids']

    vectorizer = TfidfVectorizer(
        analyzer = 'word',
        tokenizer = dummy_fun,
        preprocessor = dummy_fun,
        token_pattern = None)

    comments_tr = vectorizer.fit_transform(tokenized_comments)

    regressor = Ridge(random_state=42, alpha=0.8)
    regressor.fit(comments_tr, labels)

    less_toxic_comments = df_valid2['less_toxic']
    more_toxic_comments = df_valid2['more_toxic']

    less_toxic_comments = tokenizer(less_toxic_comments.to_list())['input_ids']
    more_toxic_comments = tokenizer(more_toxic_comments.to_list())['input_ids']

    less_toxic = vectorizer.transform(less_toxic_comments)
    more_toxic = vectorizer.transform(more_toxic_comments)

    # make predictions
    y_pred_less = regressor.predict(less_toxic)
    y_pred_more = regressor.predict(more_toxic)

    print(f'val : {(y_pred_less < y_pred_more).mean()}')

    return (y_pred_less < y_pred_more).mean()

study = optuna.create_study(direction="maximize", study_name="Best weight Finder")
func = lambda trial: objective(trial)
study.optimize(func, n_trials=50)

# Finally, after 500 trial research, you will get best weight!
Best trial: {'obscene': 1.2537639596777044, 'toxic': 1.284033992994945, 'threat': 0.3220098913216658, 'insult': 1.4594296267520046, 'severe_toxic': 0.873410475008243, 'identity_hate': 0.7399081551266039}

In [ ]:
#Result
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

# If you want to get visualization result, maybe you should use colab

In [ ]:
#plot_optimization_histor: shows the scores from all trials as well as the best score so far at each point.
# optuna.visualization.plot_optimization_history(study)

In [ ]:
#plot_parallel_coordinate: interactively visualizes the hyperparameters and scores
# optuna.visualization.plot_parallel_coordinate(study)

# Don'use Scaling trick, since that it will overfit LB score(5% data)

In [ ]:
# df_test = data
# for i in range(0, 500):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 1.25
# for i in range(801, 1200):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 1.35
# for i in range(1701, 2300):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 0.9
# for i in range(2501, 2980):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 0.90    
# for i in range(3001, 4000):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 1.35    
# for i in range(4001, 4500):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 1.4   
# for i in range(4501, 4940):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 0.95
# for i in range(5501, 5980):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 0.9
# for i in range(6001, 6500):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 1.4
# for i in range(7001, 7536):
#     df_test['score'].iloc[i] = df_test['score'].iloc[i] * 1.38  

In [ ]:
# df_test["score"] = rankdata( df_test["score"], method='ordinal')
# df_test["score"].to_csv('./submission.csv')